In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

from mpl_toolkits.mplot3d import Axes3D

# Initial Conditions and Definitions
#### Follows from example 3.4.1 in Ye

In [ ]:
# Income at time t, should be i(t)
def y(t):
    return 50000*np.exp(.03*t)

# Non-risky Investment rate
r = .04

# Continuous process (think average return rate of risky investment)
mu = .09

# Standard deviation of risky investment
sigma = .18

# Utility discount rate
rho = .03

# Hazard Function (probability of death at this moment in time)
def lamb(t):
    return 1/200 + 9/8000*t

# Insurance premium-payout ratio
def eta(t):
    return 1/200 + 9/8000*t

# Relative risk aversion
gamma = -3

# Direct Solution
### Requires a few more function definitions

In [ ]:
def integral(f, t0, tf):
    # f is a function
    return quad(f, t0, tf)[0]

# Integral function doesn't play nice with sums of constants and functions
def sum_r(t):
    return r+eta(t)
def sum_rho(t):
    return lamb(t)+rho

def H(v):
    return sum_rho(v)/(1-gamma) - .5*gamma*((mu-r)/((1-gamma)*sigma))**2 - \
                                gamma/(1-gamma)*sum_r(v)

def K(s):
    return (lamb(s)**(1/(1-gamma)))/eta(s)**(gamma/(1-gamma)) + 1

def e(t):
    return np.exp(-integral(H,t,T)) + integral(lambda s: np.exp(-integral(H,t,s))*K(s),t,T)
    
def a(t):
    return np.exp(-rho*t)*e(t)**(1-gamma)

def b(t):
    return integral(lambda s: y(s)*np.exp(-integral(sum_r,t,s)),t,T)

def c(t):
    return (1/e(t))*(x+b(t))

def D(t):
    return (lamb(t)/eta(t))**(1/(1-gamma))*1/e(t)

def Z(t):
    return D(t)*(x+b(t))

def theta(t):
    return ((mu-r)/((1-gamma)*sigma**2))*(x+b(t))

def p(t):
    return eta(t)*((D(t)-1)*x+D(t)*b(t))

In [ ]:
def V(t,x):
    return a(t)/gamma*(x+b(t))**gamma

# The Finite Difference Approach

In [ ]:
import scipy.optimize as opt

In [ ]:
# These utility definitions come in 4.3, pg 69
# Utility of consumption c 
def U(c,t):
    return np.exp(-rho*t)/gamma * c**gamma

# Utility of bequeathance Z, if dead early
def B(Z,t):
    return np.exp(-rho*t)/gamma * Z**gamma

# Utility of wealth remaining after T
def L(x,T):
    return np.exp(-rho*T)/gamma * x**gamma

### This approach will consume too much memory. Iterating recursively we will reuse too many values.

In [ ]:
# These are the 'Transition Probabilities' on pg 68.
# 3 fumctions are created for ease of substitution
# Because there is no dependence on u, I will just make them functions of t, theta_hat, c_hat, Z_hat for optimization

def Pplus(theta_hat,t):
    return delta/h*(r+eta(t)+theta_hat*(mu-r)) + (delta/(2*h**2))*(sigma**2)*(theta_hat**2)

def Pminus(theta_hat, c_hat, Z_hat, t):
    return (delta/(2*h**2))*(sigma**2)*(theta_hat**2) + (delta/(2*h))*(sigma**2)*(theta_hat**2) + (delta/h)*(c_hat+eta(t)*Z_hat)

def P(theta_hat, c_hat, Z_hat, t):
    return 1-Pplus(theta_hat,t)-Pminus(theta_hat, c_hat, Z_hat, t)

# Think of a means to save V_hat values to avoid repetition, significantly reduce the # of computations
def V_hat(t, u, T):
    # This will be a recursive function. It should terminate at t=T
    if t==T:
        return L(np.exp(u),T)
    return None
    
#     # These bounds are determined in (4.5) on pg 68
#     bounds = ((None,None), (0,None), (1-np.exp(-u)*b(t),None))
    
#     # The guess for theta-hat is .3, since that is about what we saw in Figure 9 on pg 61 for the closed form solution.
#     guess = (.3,.1,1-np.exp(u)*b(t))
#     f = lambda theta_hat, c_hat, Z_hat: \
#             -Pplus(theta_hat,t)*V_hat(t+delta,u+h,T) +\
#             -P(theta_hat,c_hat,Z_hat,t)*V_hat(t+delta,u,T) +\
#             -Pminus(theta_hat,c_hat,Z_hat,t)*V_hat(t+delta,u-h,T) +\
#             -delta*lamb(t)*B(np.exp(u)*Z_hat,t) +\
#             -delta*U(np.exp(u)*c_hat,t)
    
#     v_hat = opt.minimize(f,guess,bounds=bounds).fun
#     return -1/(1+delta*lamb(t))*v_hat

### Instead, we know what's supposed to happen at the boundary, $V(T,u)=L(e^u)$, so start at $t=T$ and work back to $t_0$.

In [ ]:
u_ = 14 # This is the u that we are interestedf in for comparison purposes
v_ = -15
# This is another value of u to test

T = 40 # represents the final time
u_min = np.round(np.log(3e-7)) # starting u at t=0, u~-15
u_max = np.round(np.log(3e6)) # ending u at t=0, u~15

delta = .001 # time step, changed from .01
h = .01 # wealth (u) step, changed from .02

diff_0 = u_max - u_min
u_0 = np.linspace(u_min, u_max, int(diff_0/h)+1) # array of u values at t=0

steps = T/delta*h

u_min -= steps # new starting u at t=T, u~-95 if delta = .01, u~-815 if delta = .001
u_max += steps # new ending u at t=T, u~95 if delta = .01, u~815 if delta = .001

diff = u_max-u_min

u_new = np.linspace(u_min, u_max, int(diff/h)+1) # array of u values at t=T

In [ ]:
u_ref = int((u_ - u_min)/h) # This is the index of the u we are interested in
v_ref = int((v_ - u_min)/h)

u_first = int((u_0[0] - u_min)/h)

start = v_ref
end = -u_first

In [ ]:
# Define the optimized happiness V at t=T
V_T = L(np.exp(u_new),T)

# Next we want to move backward one step in time from T to T-delta, using the iterative process in 4.5
t = T - delta

def V_tilda(t,w):
    return a(t)/gamma*(w)**gamma

def V_hat_exact(t,u):
    return a(t)/gamma*np.exp(u)**gamma

V_exact = np.log(-1*(V_hat_exact(t, u_new[start:end])))

# Constrained Version

In [ ]:
# Define the function to minimize with parameters strictly in theta_hat, c_hat, Z_hat
def f(params, u, t, T, sign=1.0):
    theta_hat, c_hat, Z_hat = params
    return np.log(-1*(
        1/(1+delta*lamb(t))* \
        (Pplus(theta_hat,t)*V_hat(t+delta,u+h,T) +\
        P(theta_hat,c_hat,Z_hat,t)*V_hat(t+delta,u,T) +\
        Pminus(theta_hat,c_hat,Z_hat,t)*V_hat(t+delta,u-h,T) +\
        delta*lamb(t)*B(np.exp(u)*Z_hat,t) +\
        delta*U(np.exp(u)*c_hat,t))
    ))

cons = ({'type': 'ineq', 'fun': lambda x: Pplus(x[0], t)},
        {'type': 'ineq', 'fun': lambda x: Pminus(x[0], x[1], x[2], t)},
        {'type': 'ineq', 'fun': lambda x: P(x[0], x[1], x[2], t)})

ftol = 1e-20 # This is the accuracy we want in the function f before it stops iterating.

sol = [opt.minimize(f, x0 = (.1,.2,1), args=(u, t, T), constraints=cons, method='SLSQP',# tol=1e-12,
                    options={'eps':1e-6, 'maxiter':100, 'ftol':ftol},
                  bounds = ((0,1), (0,1), (1-np.exp(-u)*b(t),None))) for u in u_new[start:end]]

V_T_1 = [s.fun for s in sol]
V_T_1_params = [s.x for s in sol]

In [ ]:
theta_hat, c_hat, Z_hat = V_T_1_params[0]
print(Pplus(theta_hat,t))
print(Pminus(theta_hat,c_hat,Z_hat,t))
print(P(theta_hat,c_hat,Z_hat,t))

In [ ]:
print(len(V_exact))
print(V_exact[:10])

In [ ]:
print(len(V_T_1))
print(V_T_1[:10])

In [ ]:
V_T_1_params[:10]

In [ ]:
plt.plot(u_new[start:end], V_T_1, label='approx', linewidth=4)
plt.plot(u_new[start:end], V_exact, label='exact', color='red')
plt.legend()
plt.show()

In [ ]:
th = [s[2] for s in V_T_1_params]
plt.plot(u_new[start:end], th, linewidth=4, label='approx $\hat Z^*$')
plt.plot(u_new[end-450:end], 1-np.exp(-u_new[end-450:end])*b(t), color='red', label='$1-e^{-u}b(T-\delta)$')

plt.legend()
plt.xlabel('$u$')
plt.ylabel('$\hat Z$')
# plt.savefig('ConstrainedZ.jpg')
plt.show()
plt.close()

# Unconstrained Version

In [ ]:
# Define the function to minimize with parameters strictly in theta_hat, c_hat, Z_hat
def f(params, u, t, T, sign=1.0):
    theta_hat, c_hat, Z_hat = params
    return np.log(-1*(
        1/(1+delta*lamb(t))* \
        (Pplus(theta_hat,t)*V_hat(t+delta,u+h,T) +\
        P(theta_hat,c_hat,Z_hat,t)*V_hat(t+delta,u,T) +\
        Pminus(theta_hat,c_hat,Z_hat,t)*V_hat(t+delta,u-h,T) +\
        delta*lamb(t)*B(np.exp(u)*Z_hat,t) +\
        delta*U(np.exp(u)*c_hat,t))
    ))

cons = ({'type': 'ineq', 'fun': lambda x: Pplus(x[0], t)},
        {'type': 'ineq', 'fun': lambda x: Pminus(x[0], x[1], x[2], t)},
        {'type': 'ineq', 'fun': lambda x: P(x[0], x[1], x[2], t)})

sol = [opt.minimize(f, x0 = (.1,.7,.7), args=(u, t, T), constraints=cons, method='SLSQP',
                    options={'eps':1e-6, 'maxiter':100, 'ftol':ftol},
                  bounds = ((0,1), (0,1), (None,None))) for u in u_new[start:end]]

V_T_1 = [s.fun for s in sol]
V_T_1_params = [s.x for s in sol]

In [ ]:
V_T_1_params[:10]

In [ ]:
plt.plot(u_new[start:end], V_T_1, label='approx $\log(-\hat V(t=T-\delta,u))$', linewidth=4)
plt.plot(u_new[start:end], V_exact, label='exact $\log(-V(t=T-\delta,u))$', color='red')
plt.legend()
plt.xlabel('$u$')
plt.ylabel('$\log(-V)$')
# plt.savefig('Vvalues.jpg')
plt.show()
plt.close()

In [ ]:
ths = [s[2] for s in V_T_1_params]
plt.plot(u_new[start:end], ths, label='approx $\hat Z^*$')
plt.plot(u_new[start:end], 1/e(t)*np.ones(len(u_new[start:end])), label='exact $\hat Z^*$')
plt.plot(u_new[end-450:end], 1-np.exp(-u_new[end-450:end])*b(t), label='$1-e^{-u}b(T-\delta)$')
plt.legend()
plt.xlabel('$u$')
plt.ylabel('$\hat Z$')
# plt.savefig('UnconstrainedZ.jpg')
plt.show()
plt.close()